In [1]:
import bashlex # parsing library #1
import bashlint # parsing library #2
from collections import Counter, OrderedDict
from pprint import pprint

In [2]:
with open(r'nl2bash/data/bash/all.cm', encoding='utf-8') as f:
    data = f.readlines()

## Playing with `bashlex` functions

In [3]:
dir(bashlex)

['__builtins__',
 '__cached__',
 '__doc__',
 '__file__',
 '__loader__',
 '__name__',
 '__package__',
 '__path__',
 '__spec__',
 'ast',
 'errors',
 'flags',
 'heredoc',
 'parse',
 'parser',
 'parsesingle',
 'parsetab',
 'shutils',
 'split',
 'state',
 'subst',
 'tokenizer',
 'utils',
 'yacc']

In [4]:
print(data[0])
print(list(bashlex.split(data[0]))) # splitting first command to elements

top -b -d2 -s1 | sed -e '1,/USERNAME/d' | sed -e '1,/^$/d'

['top', '-b', '-d2', '-s1', '|', 'sed', '-e', '1,/USERNAME/d', '|', 'sed', '-e', '1,/^$/d', '\n']


In [5]:
tp = bashlex.parse(data[0]) # parsing first command 
print(type(tp))
len(tp)

<class 'list'>


1

In [6]:
print(type(tp[0]))
tp[0]

<class 'bashlex.ast.node'>


PipelineNode(parts=[CommandNode(parts=[WordNode(parts=[] pos=(0, 3) word='top'), WordNode(parts=[] pos=(4, 6) word='-b'), WordNode(parts=[] pos=(7, 10) word='-d2'), WordNode(parts=[] pos=(11, 14) word='-s1')] pos=(0, 14)), PipeNode(pipe='|' pos=(15, 16)), CommandNode(parts=[WordNode(parts=[] pos=(17, 20) word='sed'), WordNode(parts=[] pos=(21, 23) word='-e'), WordNode(parts=[] pos=(24, 39) word='1,/USERNAME/d')] pos=(17, 39)), PipeNode(pipe='|' pos=(40, 41)), CommandNode(parts=[WordNode(parts=[] pos=(42, 45) word='sed'), WordNode(parts=[] pos=(46, 48) word='-e'), WordNode(parts=[] pos=(49, 58) word='1,/^$/d')] pos=(42, 58))] pos=(0, 58))

In [7]:
dir(tp[0])[-5:] # only few methods

['__weakref__', 'dump', 'kind', 'parts', 'pos']

In [8]:
print(tp[0].dump()) # dump allows to observe parts in pretty way

PipelineNode(pos=(0, 58), parts=[
  CommandNode(pos=(0, 14), parts=[
    WordNode(pos=(0, 3), word='top'),
    WordNode(pos=(4, 6), word='-b'),
    WordNode(pos=(7, 10), word='-d2'),
    WordNode(pos=(11, 14), word='-s1'),
  ]),
  PipeNode(pipe='|', pos=(15, 16)),
  CommandNode(pos=(17, 39), parts=[
    WordNode(pos=(17, 20), word='sed'),
    WordNode(pos=(21, 23), word='-e'),
    WordNode(pos=(24, 39), word='1,/USERNAME/d'),
  ]),
  PipeNode(pipe='|', pos=(40, 41)),
  CommandNode(pos=(42, 58), parts=[
    WordNode(pos=(42, 45), word='sed'),
    WordNode(pos=(46, 48), word='-e'),
    WordNode(pos=(49, 58), word='1,/^$/d'),
  ]),
])


`CommandNode` - have set of specific command elements  
`PipeNode` - separators  
`WordNode` - element of CommandNode

In [9]:
print(tp[0].parts) # returns all the pipeline parts in list
tp[0].pos # displays position of this specific element in original string

[CommandNode(parts=[WordNode(parts=[] pos=(0, 3) word='top'), WordNode(parts=[] pos=(4, 6) word='-b'), WordNode(parts=[] pos=(7, 10) word='-d2'), WordNode(parts=[] pos=(11, 14) word='-s1')] pos=(0, 14)), PipeNode(pipe='|' pos=(15, 16)), CommandNode(parts=[WordNode(parts=[] pos=(17, 20) word='sed'), WordNode(parts=[] pos=(21, 23) word='-e'), WordNode(parts=[] pos=(24, 39) word='1,/USERNAME/d')] pos=(17, 39)), PipeNode(pipe='|' pos=(40, 41)), CommandNode(parts=[WordNode(parts=[] pos=(42, 45) word='sed'), WordNode(parts=[] pos=(46, 48) word='-e'), WordNode(parts=[] pos=(49, 58) word='1,/^$/d')] pos=(42, 58))]


(0, 58)

In [10]:
print(data[48]) # test on more complex command

list(bashlex.split(data[48]))

for a in `find . -name '*.py'` ; do cp "$a" "$a.cp" ; echo "Added line" > "$a" ; cat "$a.cp" >> "$a" ; rm "$a.cp" ; done



['for',
 'a',
 'in',
 "`find . -name '*.py'`",
 ';',
 'do',
 'cp',
 '$a',
 '$a.cp',
 ';',
 'echo',
 'Added line',
 '>',
 '$a',
 ';',
 'cat',
 '$a.cp',
 '>>',
 '$a',
 ';',
 'rm',
 '$a.cp',
 ';',
 'done',
 '\n']

In [11]:
tp2 = bashlex.parse(data[48])
print(len(tp2))
print(tp2[0].dump())

1
CompoundNode(list=[
  ForNode(pos=(0, 120), parts=[
    ReservedwordNode(pos=(0, 3), word='for'),
    WordNode(pos=(4, 5), word='a'),
    ReservedwordNode(pos=(6, 8), word='in'),
    WordNode(pos=(9, 30), word="`find . -name '*.py'`", parts=[
      CommandsubstitutionNode(command=
        CommandNode(pos=(10, 29), parts=[
          WordNode(pos=(10, 14), word='find'),
          WordNode(pos=(15, 16), word='.'),
          WordNode(pos=(17, 22), word='-name'),
          WordNode(pos=(23, 29), word='*.py'),
        ]), pos=(9, 30)),
    ]),
    ReservedwordNode(pos=(31, 32), word=';'),
    ReservedwordNode(pos=(33, 35), word='do'),
    ListNode(pos=(36, 115), parts=[
        CommandNode(pos=(36, 51), parts=[
          WordNode(pos=(36, 38), word='cp'),
          WordNode(pos=(39, 43), word='$a', parts=[
            ParameterNode(pos=(40, 42), value='a'),
          ]),
          WordNode(pos=(44, 51), word='$a.cp', parts=[
            ParameterNode(pos=(45, 47), value='a'),
          ]),

In [12]:
print(tp2[0].list[0].dump())

ForNode(pos=(0, 120), parts=[
  ReservedwordNode(pos=(0, 3), word='for'),
  WordNode(pos=(4, 5), word='a'),
  ReservedwordNode(pos=(6, 8), word='in'),
  WordNode(pos=(9, 30), word="`find . -name '*.py'`", parts=[
    CommandsubstitutionNode(command=
      CommandNode(pos=(10, 29), parts=[
        WordNode(pos=(10, 14), word='find'),
        WordNode(pos=(15, 16), word='.'),
        WordNode(pos=(17, 22), word='-name'),
        WordNode(pos=(23, 29), word='*.py'),
      ]), pos=(9, 30)),
  ]),
  ReservedwordNode(pos=(31, 32), word=';'),
  ReservedwordNode(pos=(33, 35), word='do'),
  ListNode(pos=(36, 115), parts=[
      CommandNode(pos=(36, 51), parts=[
        WordNode(pos=(36, 38), word='cp'),
        WordNode(pos=(39, 43), word='$a', parts=[
          ParameterNode(pos=(40, 42), value='a'),
        ]),
        WordNode(pos=(44, 51), word='$a.cp', parts=[
          ParameterNode(pos=(45, 47), value='a'),
        ]),
      ]),
      OperatorNode(op=';', pos=(52, 53)),
      CommandNode

In [13]:
print(isinstance(tp2,(list,dict))) # check of isinstance
print(isinstance(tp2[0],(bashlex.ast.node)))

True
True


In [14]:
tp2[0].list

[ForNode(parts=[ReservedwordNode(pos=(0, 3) word='for'), WordNode(parts=[] pos=(4, 5) word='a'), ReservedwordNode(pos=(6, 8) word='in'), WordNode(parts=[CommandsubstitutionNode(command=CommandNode(parts=[WordNode(parts=[] pos=(10, 14) word='find'), WordNode(parts=[] pos=(15, 16) word='.'), WordNode(parts=[] pos=(17, 22) word='-name'), WordNode(parts=[] pos=(23, 29) word='*.py')] pos=(10, 29)) pos=(9, 30))] pos=(9, 30) word="`find . -name '*.py'`"), ReservedwordNode(pos=(31, 32) word=';'), ReservedwordNode(pos=(33, 35) word='do'), ListNode(parts=[CommandNode(parts=[WordNode(parts=[] pos=(36, 38) word='cp'), WordNode(parts=[ParameterNode(pos=(40, 42) value='a')] pos=(39, 43) word='$a'), WordNode(parts=[ParameterNode(pos=(45, 47) value='a')] pos=(44, 51) word='$a.cp')] pos=(36, 51)), OperatorNode(op=';' pos=(52, 53)), CommandNode(parts=[WordNode(parts=[] pos=(54, 58) word='echo'), WordNode(parts=[] pos=(59, 71) word='Added line'), RedirectNode(heredoc=None input=None output=WordNode(parts

### Iterate until get all basic elements

Output of `bashlex.parse()` is list.  
List has elements of `bashlex.ast.node` which may have either `list`, `parts` or `command` properties (at least figured those two by now) that contain next elements.

In [15]:
def iterate_bashlex(input_object):
    output = Counter()
    if isinstance(input_object, list):
        for element in input_object:
            output += iterate_bashlex(element)
    elif isinstance(input_object, bashlex.ast.node):
        object_methods = [x for x in dir(input_object) if '__' not in x]
        if 'command' in object_methods:
            output += iterate_bashlex(input_object.command)
        elif 'list' in object_methods:
            for element in input_object.list:
                output += iterate_bashlex(element)
        elif 'parts' in object_methods:        
            if input_object.parts:
                for part in input_object.parts:
                    output += iterate_bashlex(part)
            else:
                output[input_object.word] += 1
        elif 'word' in object_methods:
            output[input_object.word] += 1
    return output

In [16]:
for nodes in tp2:
    token_dict = iterate_bashlex(nodes)
print(token_dict)

Counter({'for': 1, 'a': 1, 'in': 1, 'find': 1, '.': 1, '-name': 1, '*.py': 1, ';': 1, 'do': 1, 'cp': 1, 'echo': 1, 'Added line': 1, 'cat': 1, 'rm': 1, 'done': 1})


### Testing on full corpus

In [17]:
# testing bashlex functions with lists
alldict = Counter()
for command in data[0:10]:
    for node in bashlex.parse(command):
        alldict += iterate_bashlex(node)
print(dict(alldict))

{'top': 10, '-b': 5, '-d2': 1, '-s1': 1, 'sed': 4, '-e': 2, '1,/USERNAME/d': 1, '1,/^$/d': 1, '-n': 4, '1': 2, '-u': 1, 'abc': 1, 'awk': 3, 'NR>7 { sum += $9; } END { print sum; }': 1, '-d': 1, '5': 1, '2': 1, '$1 == "PID" {block_num++; next} block_num == 2 {sum += $9;} END {print sum}': 1, '-bn1': 3, 'grep': 3, 'zombie': 2, '/Cpu/p': 1, '{print $4" "$6" "$8" "$10}': 1, '-n1': 3, '-c': 1, 'processname': 1, 's/\\(.*\\)$/\\1__CUSTOM_LINE_MARKER/g': 1}


In [19]:
# trying on all data
alldict = Counter()
errors = 0
for i,command in enumerate(data):
    print(f'{i}/{len(data)}',end='\r')
    try:
        for node in bashlex.parse(command):
            alldict += iterate_bashlex(node)
    except bashlex.errors.ParsingError as ex:
        #print(command, ex)
        errors += 1
    except NotImplementedError as ex:
        #print(command, ex)
        errors += 1
    except TypeError as ex:
        #print(command, ex)
        errors += 1

print(f'Parsing errors encountered on {errors*100/len(data):.2f} % of data.')

Parsing errors encountered on 1.01 % of data.


In [23]:
# unique elements
len(alldict)

8724

In [25]:
# only ~2k unique elements appear more than 3 times
print(len({k:v for k,v in alldict.items() if v >= 3})) 
# only ~1k unique elements appear more than 5 times
print(len({k:v for k,v in alldict.items() if v >= 5})) 

1950
1074


In [26]:
# 20 elements with highest counts
alldict.most_common(20)

[('find', 7747),
 ('.', 3728),
 ('-name', 3563),
 ('-type', 3354),
 ('f', 2438),
 ('{}', 2274),
 ('-exec', 2086),
 (';', 1823),
 ('xargs', 1523),
 ('grep', 1333),
 ('-print', 927),
 ('d', 848),
 ('/', 778),
 ('1', 740),
 ('-print0', 725),
 ('sort', 704),
 ('-o', 667),
 ('-l', 655),
 ('sed', 611),
 ('rm', 611)]